- get 5/50/n samples per cell type: get_samples()
- save resulting df: X_test_shap_....h5ad
- prepare private test data:
 - svd
 - also apply get_samples()
 - save private_test_input_sample.h5ad

In [1]:
%%capture output
!pip install ipywidgets
!pip install --upgrade jupyter
!pip install IProgress
!pip install anndata

In [1]:
import numpy as np
import pandas as pd
import os

import pickle

import scanpy as sc
import anndata as ad

## create samples representing each cell type for public test data

### add cell_ids and cell_type to train and test data

In [2]:
lrz_path = '/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/'
index_path = lrz_path + 'open-problems-multimodal-3rd-solution/input/preprocess/cite/'
cite_feature_path = lrz_path + 'open-problems-multimodal-3rd-solution/input/features/cite/'   # '../../../input/features/cite/'

private_data_path = lrz_path + 'kaggle/full_data'

In [3]:
# train_ids = np.load(index_path + "train_cite_raw_inputs_idxcol.npz", allow_pickle=True)
# test_ids = np.load(index_path + "test_cite_raw_inputs_idxcol.npz", allow_pickle=True)

train_index = np.load(index_path + "train_cite_raw_inputs_idxcol.npz", allow_pickle=True)["index"]
# train_column = np.load(index_path + "train_cite_raw_inputs_idxcol.npz", allow_pickle=True)["columns"]  # not used
test_index = np.load(index_path + "test_cite_raw_inputs_idxcol.npz", allow_pickle=True)["index"]
print(len(list(train_index)))
print(len(list(test_index)))

70988
48203


In [4]:
metadata = pd.read_csv(lrz_path + 'neurips_competition_data/metadata.csv')
metadata_filtered = metadata[metadata['cell_id'].isin(test_index)]
metadata_filtered.index=metadata_filtered['cell_id']
metadata_filtered = metadata_filtered.drop('cell_id', axis=1)
metadata_filtered.shape   # somehow only 41187 matching cell_ids instead of 48203

(41187, 4)

In [5]:
X_train_cell_ids = pd.read_pickle(cite_feature_path  + 'X_svd_128.pickle')   # == X_svd_128 in make-features second to last cell == train set for model #16
X_train_cell_ids = ad.AnnData(X=X_train_cell_ids)
X_train_cell_ids.obs_names = train_index
# X_train_cell_ids.to_df().head()

# cell type from metadata
X_test_cell_ids = pd.read_pickle(cite_feature_path  + 'X_test_svd_128.pickle')   # == test set for model #16
X_test_cell_ids = ad.AnnData(X=X_test_cell_ids)
X_test_cell_ids.obs_names = test_index
X_test_cell_ids = X_test_cell_ids[X_test_cell_ids.obs_names.isin(metadata_filtered.index), :]
X_test_cell_ids.obs = metadata_filtered
X_test_cell_ids

/dss/dsshome1/02/di93zoj/.local/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/dss/dsshome1/02/di93zoj/.local/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 41187 × 212
    obs: 'day', 'donor', 'cell_type', 'technology'

In [6]:
X_test_cell_ids.obs['cell_type'].value_counts()

HSC     9451
MasP    9064
EryP    8788
NeuP    7719
MkP     4844
MoP     1215
BP       106
Name: cell_type, dtype: int64

### create dataset of n samples per cell type for SHAP beeswarm plot

In [8]:
def get_samples(samples_per_cell_type, data):
    ''' data: AnnData with obs 'cell_type' '''
    
    unique_types = np.unique(data.obs['cell_type'])

    sample_obs = []

    # get indices of n samples per cell type
    for t in unique_types:
        sampled_rows = data[data.obs.cell_type == t].obs.sample(n=samples_per_cell_type, random_state=42)
        sample_obs.append(sampled_rows.index) 

    # select rows in data with matching indices
    X_test_shap = data[[elem for index_obj in sample_obs for elem in index_obj.tolist()]]

    return X_test_shap

X_test_shap = get_samples(50, X_test_cell_ids)

In [9]:
os.chdir('../..')

In [10]:
# rename imp_ columns to gene ids:
handselected_84_gene_ids = np.loadtxt('2.preprocess_to_feature/cite/handselected_84_gene_ids.txt', dtype=str)

In [11]:
# X_test_shap currently has base_svd and imp columns -> rename imp (handselected genes)
X_test_shap.var_names = X_test_shap.var_names[:-84].tolist() + handselected_84_gene_ids.tolist()
print(X_test_shap.shape)
X_test_shap.to_df().head(1)

(350, 212)


,base_svd_0,base_svd_1,base_svd_2,base_svd_3,base_svd_4,base_svd_5,base_svd_6,base_svd_7,base_svd_8,base_svd_9,...,ENSG00000188404_SELL,ENSG00000124570_SERPINB6,ENSG00000235169_SMIM1,ENSG00000095932_SMIM24,ENSG00000137642_SORL1,ENSG00000128040_SPINK2,ENSG00000072274_TFRC,ENSG00000205542_TMSB4X,ENSG00000133112_TPT1,ENSG00000026025_VIM
cell_id,,,,,,,,,,,,,,,,,,,,,
5cb9daaca7ac,71.003983,13.2008,-6.168917,12.436371,-0.578827,0.227373,1.382157,0.969637,-1.477851,6.248293,...,0.0,0.0,0.0,0.0,0.0,0.947536,0.0,5.313975,4.545222,3.069516


In [12]:
X_test_shap.write("4.model/pred/X_test_shap_16_50_samples.h5ad")

## create samples representing each cell type for private test data
In this case, also need to apply SVD transform.

In [17]:
metadata = pd.read_pickle(private_data_path + '/20220830_citeseq_metadata_cells.pkl')
metadata.shape

(119191, 4)

In [18]:
train = pd.read_pickle(private_data_path + '/20220830_citeseq_rna_count_train.pkl')
train.shape

(70988, 22085)

In [19]:
private_test_input = pd.read_pickle(private_data_path + '/20220830_citeseq_rna_count_test_input_private.pkl') # already has gene names (ordered alphabetically)
private_test_input.shape

(26867, 22085)

In [20]:
private_test_target = pd.read_pickle(private_data_path + '/20220830_citeseq_prot_count_test_labels_private.pkl')
private_test_target.shape

(26867, 140)

In [21]:
# filter metadata to get matching cells  -> then use as observation to annotate data with cell_type
metadata_filtered = metadata[metadata.index.isin(private_test_input.index)]
metadata_filtered.head()

,kaggle_dataset,day,donor,cell_type
"CATAAGCAGCATGATA-1-('27678', 7)",test_private,7,27678,MasP
"GGTGATTGTATGAAAC-1-('27678', 7)",test_private,7,27678,HSC
"ATAGGCTAGCTCTGTA-1-('27678', 7)",test_private,7,27678,MasP
"TCATGAGGTGGATTTC-1-('27678', 7)",test_private,7,27678,MasP
"TAAGCGTTCGGCTGGT-1-('27678', 7)",test_private,7,27678,HSC


In [22]:
private_test_input = ad.AnnData(private_test_input, obs=metadata_filtered)
private_test_target = ad.AnnData(private_test_target, obs=metadata_filtered)
private_test_input

AnnData object with n_obs × n_vars = 26867 × 22085
    obs: 'kaggle_dataset', 'day', 'donor', 'cell_type'

In [23]:
private_test_input.write(lrz_path + "/large_preprocessed_files/private_test_input.h5ad")
private_test_target.write(lrz_path + "/large_preprocessed_files/private_test_target.h5ad")

### preprocess input data - similar to make-base-feature.py

In [24]:
# all_genes = np.loadtxt('2.preprocess_to_feature/cite/all_genes_names.txt', dtype=str)  # not used
# all_genes   # 22001 genes sorted alphabetically

In [25]:
# indices of 84 handselected genes
use_imp_cols_84 = np.loadtxt('2.preprocess_to_feature/cite/use_imp_cols_84.txt')
use_imp_cols_84

array([ 3290.,  4010.,  4743.,  4841.,  5172.,  5260.,  5619.,  5632.,
        5939.,  6120.,  6699.,  6704.,  6705.,  6710.,  6713.,  6714.,
        6716.,  6720.,  6733.,  6734.,  6735.,  6737.,  6741.,  6744.,
        6749.,  6750.,  6756.,  6759.,  6761.,  6762.,  6763.,  6768.,
        6771.,  6774.,  6777.,  7292.,  7338.,  7616.,  7688.,  7772.,
        8238.,  8747.,  9331.,  9602.,  9908.,  9948., 10500., 10588.,
       10589., 10602., 10603., 10606., 10732., 10904., 10937., 10956.,
       11309., 11398., 11597., 11662., 13011., 13390., 13801., 14785.,
       15124., 15266., 15571., 16503., 17087., 17138., 17323., 17339.,
       17512., 17514., 17750., 17826., 18436., 18449., 18660., 18762.,
       19495., 19923., 20095., 20883.])

In [26]:
# indices of other 22001 genes
use_cols_84 = np.loadtxt('2.preprocess_to_feature/cite/use_cols_84.txt')
len(use_cols_84)

22001

In [27]:
df_use = private_test_input[:, use_cols_84]
# df_use = private_test_input_raw[:, use_cols_84]  # _raw
#print(df_use.X)

sc.pp.normalize_per_cell(df_use)
sc.pp.log1p(df_use.X)

with open('2.preprocess_to_feature/cite/svd_model_fitted_128.pkl', 'rb') as file:
    svd = pickle.load(file)
    
result_svd = svd.transform(df_use.X)

test_cite_pp = pd.DataFrame(result_svd, index = private_test_input.obs_names)     # has cell_ids   # _raw

# important protein cols ---------------------------------------------------------------------

df_important = private_test_input.X[:, use_imp_cols_84]   # _raw
df_important = sc.AnnData(X = df_important)
sc.pp.normalize_per_cell(df_important)
sc.pp.log1p(df_important)

test_imp = pd.DataFrame(df_important.X.toarray(), index = private_test_input.obs_names).add_prefix('imp_')   # _raw

/usr/local/lib/python3.8/dist-packages/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell


In [28]:
test_cite_pp = test_cite_pp.add_prefix('base_svd_')
test_cite_pp.shape

(26867, 128)

In [29]:
test_imp.shape

(26867, 84)

### preprocess input data - similar to make-features.py

In [34]:
X_test_svd_128 = pd.concat([test_cite_pp.reset_index(drop=True), 
                            test_imp.reset_index(drop=True),
                           ], axis = 1)
X_test_svd_128.head(3)   # final X_test

,base_svd_0,base_svd_1,base_svd_2,base_svd_3,base_svd_4,base_svd_5,base_svd_6,base_svd_7,base_svd_8,base_svd_9,...,imp_74,imp_75,imp_76,imp_77,imp_78,imp_79,imp_80,imp_81,imp_82,imp_83
0,80.546516,8.945739,30.409332,-4.856643,-2.094273,-9.574170,-6.450809,2.747784,-2.683906,5.604310,...,1.696244,1.696244,0.0,0.000000,1.571028,0.000000,1.696244,2.253848,2.238665,2.178289
1,83.140480,-4.495127,9.941430,-12.366527,5.982366,-11.163328,4.029469,-4.822271,1.670215,4.221057,...,1.653121,1.831568,0.0,1.831568,1.803997,1.803997,1.522853,2.038796,2.211638,2.038796
2,61.207283,24.319401,16.274017,-4.918456,7.281606,-3.375170,-1.265758,-8.099415,-6.601540,2.202348,...,2.011695,2.011695,0.0,0.000000,0.000000,0.000000,0.000000,2.380579,2.486660,2.333222


In [31]:
with open('4.model/pred/private_X_test_svd.pkl', 'wb') as f:   # _raw
    pickle.dump(X_test_svd_128, f)

### get sample per cell type for SHAP

In [32]:
X_test_svd_128_ann = ad.AnnData(X=X_test_svd_128, obs=metadata_filtered)
X_test_svd_128_ann

AnnData object with n_obs × n_vars = 26867 × 212
    obs: 'kaggle_dataset', 'day', 'donor', 'cell_type'

In [35]:
private_test_input_sample = get_samples(5, X_test_svd_128_ann)

In [36]:
private_test_input_sample.obs

,kaggle_dataset,day,donor,cell_type
"CTCCCTCGTATGAAAC-1-('32606', 7)",test_private,7,32606,BP
"CATGCAATCGAAATCC-1-('27678', 7)",test_private,7,27678,BP
"AAGCGAGGTTCATCGA-1-('31800', 7)",test_private,7,31800,BP
"TTCGCTGAGAAAGTCT-1-('27678', 7)",test_private,7,27678,BP
"TCCAGAACATCTCATT-1-('31800', 7)",test_private,7,31800,BP
"GGAGATGTCTGAGAAA-1-('32606', 7)",test_private,7,32606,EryP
"CTTCCTTCACAAACGG-1-('27678', 7)",test_private,7,27678,EryP
"ACTATGGTCATGGTAC-1-('27678', 7)",test_private,7,27678,EryP
"GTGCACGCAACCTATG-1-('13176', 7)",test_private,7,13176,EryP
"CCTCACACACAAGTGG-1-('27678', 7)",test_private,7,27678,EryP


In [38]:
private_test_input_sample.write("4.model/pred/private_test_input_sample.h5ad")